# Biostat M280 Homework 4
Sarah Ji

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


## Q1 Solution: 
 
>First we should plug in the pi(p) formula in the f(x) formula and then combine like terms.

 $$
\int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p} 
= \int_{\Delta_d}\binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}d \mathbf{p}
$$
 
>Then we can use the kernel trick to integrate the first pdf to 1 and change the coefficients accordingly.
 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} d\mathbf{p}
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}\frac{\Gamma(|\alpha|+|\mathbf{x}|)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}\int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

> Note this is a density of a Dirchlet pdf and so the integral evalutes to 1, giving us: 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)} 1
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$

$$
= f(\mathbf{x} \mid \alpha)
$$

## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

## Q2 Solution:

The log likelihood is a non-concave function. 

In [3]:
using SpecialFunctions
# if second derivative of the log likelihood is not positive definite then it is not concave
α_test = [1, 2]
x_test = [17 , 38]
term1 = diagm(polygamma.(1, α_test) .- polygamma.(1, α_test + x_test))
Neg_Hessian = term1 .- (polygamma.(1, sum(α_test)) .- polygamma.(1, sum(α_test) + sum(x_test)))#this gives a non-pd matrix meaning it's not concave.

2×2 Array{Float64,2}:
  1.21026   -0.377543
 -0.377543   0.242076

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

## Q3 Solution:

In [5]:
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    term1 = 0.0
    Logpdf = 0.0
    for i in 1:length(x)
        term1 += lgamma(α[i] + x[i]) - lgamma(α[i]) - lfact(x[i])
    end
    
    sumalpha = sum(α)
    sumx = sum(x)
    Logpdf = term1 + lfact(sumx) + lgamma(sumalpha) - lgamma(sumx + sumalpha)
    return Logpdf
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

## Q4 Solution:

In [20]:
traindata = readcsv("optdigits.tra", Int)
digit = traindata[:, end]
counts = traindata[:, 1:64]'

64×3823 Array{Int64,2}:
  0   0   0   0   0   0   0   0   0  …   0   0   0   0   0   0   0   0   0
  1   0   0   0   0   0   0   0   0      0   0   1   0   0   0   0   0   0
  6  10   8   0   5  11   1   8  15      9   9  10   6   5   0   3   6   2
 15  16  15   3  14  16  11  10   2     16  16  16  16  13   1  15  16  15
 12   6  16  11   4  10  13   8  14      6  12  16  11  11  12   0   2  16
  1   0  13  16   0   1  11   7  13  …   0   1   4   0   2   1   0   0  13
  0   0   0   0   0   0   7   2   2      0   0   0   0   0   0   0   0   1
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0      0   0   0   0   0   0   0   0   0
  7   7   1   0   0   4   0   1   0      2   3   8   1   2   0   0   0   0
 16  16  11   5  13  16   9  15  16  …  14  16  13  16  15   0  11  15   3
  6   8   9  16   8  10  14  14  15     16  10  12   2   6  14  14  10   7
  6  16  11  11   0  15   6  12  12     16  15  16  12   5  10   0   0  10
 

In [21]:
digit

3823-element Array{Int64,1}:
 0
 0
 7
 4
 6
 2
 5
 5
 0
 8
 7
 1
 9
 ⋮
 1
 4
 4
 8
 9
 3
 9
 9
 4
 6
 6
 7

In [7]:
eachlogpdf = dirmult_logpdf(counts, ones(64))

3823-element Array{Float64,1}:
 -165.188
 -176.23 
 -167.774
 -165.564
 -157.79 
 -176.071
 -158.423
 -159.258
 -174.302
 -178.407
 -171.294
 -169.383
 -175.753
    ⋮    
 -160.49 
 -158.633
 -156.935
 -171.975
 -177.638
 -169.735
 -158.423
 -159.465
 -159.877
 -169.735
 -173.149
 -155.411

Note that each observation is independent, so the overall log likelihood is the sum of each of the log pdf's in the `eachlogpdf` vector above.

In [8]:
Log_Likelihood = sum(eachlogpdf)


-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

## Q5 Solution:

>Let $\psi(x) = \frac{d}{dx}[ \ln \Gamma(x)]$ be the digamma function of x.
Let $\psi_1(x) = \frac{d}{dx}[\psi(x)]$ be the trigamma function of x.

>The Score function, $\nabla L(\alpha) = \frac{d}{d\alpha_j}L(\alpha) = 0 + \sum_{i=1}^n[\psi(\alpha_j + x_{ij}) - \psi(\alpha_j)] - \sum_{i=1}^n[\psi(|\alpha| + |x_i|) - \psi(|\alpha|)]$


>The observed information matrix $-d^2L(\alpha)$ has different forms on the diagonals and on the off diagonals.
<br>
>On the diagonals (where i = j):
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \sum_{i=1}^n[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})] - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$
<br>


>On the off diagonals (where i $\ne$ j)
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$

>Now we can see that the observed information matrix has a special structure of a diagonal plus an easy low-rank matrix which is rank 1.
$$
-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \mathbf{D} - c\mathbf{1}\mathbf{1}^T
$$

$$
d_j = \sum_{i=1}^n\Big[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})\Big]
$$

$$
c = \sum_{i=1}^n\Big[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|) \Big]
$$

<br>
>The Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ is difficult to compute, which highlights why Fisher scoring method is inefficient for computing MLE in this example. 

$$
-E\Big[\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) \Big] = E[\mathbf{D} - c\mathbf{1}\mathbf{1}^T]
$$

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

## Q6 Solution:

No, the observed information matrix is not always positive definite. Some remedies we can make to it involve taking the expectation, resulting in swapping out the observed information matrix for the expected information matrix (Fisher Scoring). However, this remedy may not be the best case in this scenario, as taking expectation of the observed information matrix may be difficult. Thus, we use the Sherman Morrison formula from HW 1 to use the special structure of the observed information matrix, a diagonal plus a rank one update. 

>The observed information matrix has a special structure, a diagonal matrix plus a low rank matrix so we can use the Woodbury formula for the evaluation of the Newton direction. 

>In the case where the matrix fails to be diagonal we can add a positive matrix $\eta \mathbf{I}$, where $\eta$ > 0 to make it become positive definite.  

>From the Sherman-Morrison formula and the determinant formula from HW1:

$$
det(\mathbf{D} - c\mathbf{1}\mathbf{1}^T) = det(\mathbf{D})det(\mathbf{I} + c\mathbf{1}^T\mathbf{D}^{-1}\mathbf{1})
$$

$$
= (\prod_{i = 1}^d d_j)(1 - c \sum^d_{j=1}d_j^{-1})
$$
> In order to have a positive definite matrix, the diagonals and the determinant must be positive! So we check that:
$(1 - c \sum^d_{j=1}d_j^{-1}) > 0$


$$
1 > c\sum^d_{j=1}d_j^{-1} \implies c < \cfrac{1}{\sum^d_{j=1}d_j^{-1}}
$$


>We find that the observed information matrix is positive definite when $c < \cfrac{1}{\sum_{i = 1}^d d_j^{-1}}$.

## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)

### Q7 Solution:

Let $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$. Then, we can begin by noting the first and second moments of this distribution. 

>The first moment of the Dirichlet Distribution is:
$$E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d$$
<br>

>The second moment of the Dirichlet Distribution is:
$$E[P_j^2] = \frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha| + 1)}, 1 \le j \le d$$
<br>

Since we know $E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d,$ we know that $\alpha_j = E[P_j]*|\alpha|.$ Now we will use these two moments to first find the value of $\hat|\alpha|$ to find a good estimate for the starting value $\hat\alpha_j$
<br>

>Let $Z = \sum_{j=1}^d \frac{E[P_j^2]}{E[P_j]},$ then we have:

$$Z = \sum_{j = 1}^d \cfrac{E(p_j^2)}{E(p_j)} = \sum_{j = 1}^d \cfrac{\frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha|+ 1)}}{\frac{\alpha_j}{|\alpha|}} = \sum_{j=1}^d \frac{\alpha_j + 1}{|\alpha| + 1} = \frac{|\alpha| + \sum_{j=1}^d 1}{|\alpha| + 1} =  \cfrac{|\alpha| + d}{|\alpha| + 1}$$
<br>
>Now we want to solve for $|\alpha|$:
<br>
$$ Z(|\alpha| + 1) = |\alpha| + d$$
<br>
$$Z|\alpha| + Z = |\alpha| + d$$
<br>
$$Z|\alpha| - |\alpha| = d - Z$$
<br>
$$|\alpha|(Z - 1) = d - Z$$
<br>
$$|\alpha| = \frac{d - Z}{Z - 1}$$
<br>

> Recall that we are estimating Z by $\hat{Z}$:

$$\hat{Z} = \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}$$
<br>

>Now we can estimate $\hat{|\alpha|}$ using the estimates for $\hat{Z}$:
<br>

$$\hat{|\alpha|} = \frac{d - \hat{Z}}{\hat{Z} - 1}
$$
<br>

$$ = \frac{d - \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}}{\sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})} - 1}
$$

## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

In [2]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
function dirmult_newton(X::Matrix; α0::Vector = nothing, 
            maxiters::Int = 100, tolfun::Float64 = 1e-6)
    n, d = size(X)
    #first find the rows where there are no observations
    non_zero_X = find(sum(X, 1) .> 0.0) 
    #get rid of columns with all 0's
    X = copy(X[:, nonzeroobs])
    #create a copy of X transpose
    Xt = copy(X.')
    
    # set default starting point from Q7
    α0 = initial_alpha(X)
    
    sumX = sum(X, 2)
    n_2, d_2 = size(X)
    digamma_alpha = zeros(d_2)
    score = zeros(d_2)
    αhat = copy(α0)
    newton_direction = zeros(d_2)
    trigamma_sum_alpha = 0.0
    loglold = sum(dirmult_logpdf(Xt, αhat))
    # Newton loop
    for iter in 1:maxiters
        sumα = sum(αhat)
        score = zeros(d_2)
        # evaluate gradient (score)
        digamma_alpha = digamma(αhat)
        digamma_sum_alpha = digamma(sumα)
        for j in 1:d
            for i in 1:n
                score[j] += digamma(αhat[j] + X[i, j]) - digamma_alpha[j] - digamma(sumα + sumX[i]) + digamma_sum_alpha
            end
        end
        # compute Newton's direction
        
        # line search loop
        for lsiter in 1:10
            # step halving
        end
        
        # check convergence criterion
        if abs(logl - loglold) < tolfun * (abs(loglold) + 1)
            break;
        end
    end
    
    # compute logl, gradient, Hessian from final iterate
    
    # output
    
end

dirmult_newton

In [17]:
find(sum(x, 1) .> 0.0) 

2-element Array{Int64,1}:
 1
 3

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

In [18]:
O = readdlm("optdigits.tra", ',')

3823×65 Array{Float64,2}:
 0.0  1.0   6.0  15.0  12.0   1.0   0.0  …  14.0   7.0   1.0   0.0  0.0  0.0
 0.0  0.0  10.0  16.0   6.0   0.0   0.0     16.0  15.0   3.0   0.0  0.0  0.0
 0.0  0.0   8.0  15.0  16.0  13.0   0.0     14.0   0.0   0.0   0.0  0.0  7.0
 0.0  0.0   0.0   3.0  11.0  16.0   0.0      1.0  15.0   2.0   0.0  0.0  4.0
 0.0  0.0   5.0  14.0   4.0   0.0   0.0     12.0  14.0   7.0   0.0  0.0  6.0
 0.0  0.0  11.0  16.0  10.0   1.0   0.0  …  16.0  16.0  16.0  16.0  6.0  2.0
 0.0  0.0   1.0  11.0  13.0  11.0   7.0     13.0   5.0   0.0   0.0  0.0  5.0
 0.0  0.0   8.0  10.0   8.0   7.0   2.0     13.0   8.0   0.0   0.0  0.0  5.0
 0.0  0.0  15.0   2.0  14.0  13.0   2.0     12.0   5.0   0.0   0.0  0.0  0.0
 0.0  0.0   3.0  13.0  13.0   2.0   0.0     15.0  11.0   6.0   0.0  0.0  8.0
 0.0  0.0   6.0  14.0  14.0  16.0  16.0  …  12.0   0.0   0.0   0.0  0.0  7.0
 0.0  0.0   0.0   3.0  16.0  11.0   1.0      2.0  14.0  14.0   1.0  0.0  1.0
 0.0  0.0   0.0   4.0  13.0  16.0  16.0      5.0  

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q10 Solution:

Dirichlet Multinomial is better for all digits.

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.

test set to get the prior probability distribution for p and then multiply each DMloglikelihood to get posterior distribution of each digit, plug in new image to this posterior to do classification